<img src = "imgs/TeS_img.png">

In [1]:
import numpy as np
import pyomo.environ as pyo
from scipy.integrate import quad

# 1. Defining components and properties
components = ['H2O', 'CH4', 'CO2', 'CO', 'H2']

# Cp/R = A + B T + C T^2 + D T^-2
CP_data = [
    [3.47, 0.00145, 0, 12100],            # H2O
    [1.702, 0.009081, -0.000002164, 0],   # CH4
    [5.457, 0.001045, 0, -115700],        # CO2
    [3.376, 0.000557, 0, -3100],          # CO
    [3.249, 0.000422, 0, 8300]            # H2
]

# ΔHf°, ΔGf° (J/mol) - Standard formation enthalpy and Gibbs free energy
prop_form = [
    [-241818, -228572],   # H2O
    [-74520,  -50460],    # CH4
    [-393509, -394359],   # CO2
    [-110525, -137169],   # CO
    [0, 0]                # H2
]

# Stoichiometric matrix (components x elements: C, H, O)
A = np.array([
    [0, 2, 1],  # H2O
    [1, 4, 0],  # CH4
    [1, 0, 2],  # CO2
    [1, 0, 1],  # CO
    [0, 2, 0]   # H2
])

In [2]:
# 2. Calculating standard chemical potential

def calculate_standard_chemical_potential(T, prop_form, CP_data):
    """Calculate μ°_i(T) from ΔHf°, ΔGf° and Cp(T)."""
    R = 8.314  # Universal gas constant (J/mol/K)
    T0 = 298.15  # Reference temperature (K)
    mu = []

    for (dH, dG), (Acp, Bcp, Ccp, Dcp) in zip(prop_form, CP_data):
        def cp(Tp):  # Heat capacity in J/mol/K
            return R * (Acp + Bcp*Tp + Ccp*Tp**2 + Dcp/Tp**2)

        def inner(Tp):
            # Integral of Cp from T0 to Tp
            val, _ = quad(cp, T0, Tp)
            return (dH + val) / Tp**2

        # Calculate the temperature-dependent integral
        integ, _ = quad(inner, T0, T)
        mu_i = T * (dG/T0 - integ)
        mu.append(mu_i)

    return mu  # List of μ°_i(T) for each component

In [ ]:
# 3. Gibbs energy minimization model
# In this case, we will use the Pyomo library to set up and solve the optimization problem.
# The components are considered as ideal gases.

def solve_gibbs_minimization(initial_moles, T, P):
    """
    Minimize G(T,P) for ideal gas phase with φ=1.
    P in bar (consistent with use in ln(P)). 
    """
    n_components = len(initial_moles)
    initial_atoms = A.T @ np.array(initial_moles)

    # Constant parameters   
    R = 8.314  # Gas constant (J/mol/K)
    mu_standard = calculate_standard_chemical_potential(T, prop_form, CP_data)
    epsilon = 1e-12   # Regularization to avoid log(0)
    phi = np.ones(n_components)  # Can be replaced with updated values in external loop

    # Model
    model = pyo.ConcreteModel()
    model.I = range(n_components)

    # Decision variables (moles)
    # bounds (ε, None) avoids log(0) and problems at initialization
    model.n = pyo.Var(model.I, domain=pyo.NonNegativeReals, bounds=(epsilon, None))
    for i in model.I:
        model.n[i].set_value(max(initial_moles[i], epsilon))

    # Objective: G = Σ n_i [ μ°_i(T) + RT ln(φ_i * y_i * P) ]
    def obj_rule(m):
        n_tot = sum(m.n[i] for i in m.I)
        # y_i ≈ (n_i + ε) / (n_tot + N*ε) -> smooth and well-defined
        denom = n_tot + len(m.I)*epsilon
        return sum(
            (mu_standard[i] + R*T*(pyo.log(phi[i]) + pyo.log((m.n[i] + epsilon)/denom) + pyo.log(P))) * m.n[i]
            for i in m.I
        )
    model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

    # Element balance: A^T n = A^T n0
    model.elcon = pyo.ConstraintList()
    for k in range(A.shape[1]):  # For each element (C,H,O)
        target = float(initial_atoms[k])
        model.elcon.add(sum(A[i, k]*model.n[i] for i in model.I) == target)

    # Solver
    solver = pyo.SolverFactory('ipopt')
    solver.options['tol'] = 1e-8
    solver.options['max_iter'] = 3000
    solver.options['print_level'] = 1

    results = solver.solve(model, tee=False)

    n_sol = np.array([pyo.value(model.n[i]) for i in model.I])
    return n_sol, results

In [4]:
# 4. Solving the optimization problem
if __name__ == "__main__":
    initial_moles = [1.0, 0.5, 0.0, 0.0, 0.0]  # H2O, CH4, CO2, CO, H2
    T = 1000  # K
    P = 1     # bar

    print("Gibbs Energy Minimization for Chemical Equilibrium")
    print("=" * 60)
    print(f"Initial composition: {dict(zip(components, initial_moles))}")

    n_final, res = solve_gibbs_minimization(initial_moles, T, P)
    print("Status:", res.solver.termination_condition)
    print("Final moles:", dict(zip(components, n_final)))

Gibbs Energy Minimization for Chemical Equilibrium
Initial composition: {'H2O': 1.0, 'CH4': 0.5, 'CO2': 0.0, 'CO': 0.0, 'H2': 0.0}
Status: optimal
Final moles: {'H2O': np.float64(0.39326228404633917), 'CH4': np.float64(0.02180293019544616), 'CO2': np.float64(0.12854064614910699), 'CO': np.float64(0.34965642365544686), 'H2': np.float64(1.5631318555627685)}
